## Harjoitustyö

## Vaihe 1

#### Tehtävä 1. Hae Digitrafficin REST-rajapintapalvelusta (JSON-formaatissa) aktiiviset junat pääteosoitteella `/live-trains` Helsingin asemalta, joiden hakuparametrit ovat seuraavat:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False


#### Tehtävä 2. Tämän jälkeen lue kyseinen data Pandasin dataframeen ilman muokkaamista, jolloin sarakkeiden otsikot ovat seuraavanlaiset:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version` 

In [2]:
import requests

url = "http://rata.digitraffic.fi/api/v1"
endpoint = "/live-trains/"
station = "station/HKI"
params = "?arrived_trains=10&departed_trains=10&include_nonstopping=false"

tr_url = "{0}{1}{2}{3}".format(url, endpoint, station, params)
tr_req = requests.get(tr_url)
tr_req.status_code

200

In [3]:
import pandas as pd
train_data = tr_req.json()
df = pd.DataFrame(train_data)
df.head(3)

,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
0,68,2021-03-29,10,vr,IC,Long-distance,,False,False,277557820543,REGULAR,2021-02-19T10:43:08.000Z,"[{'stationShortCode': 'KUO', 'stationUICCode':..."
1,265,2021-03-29,10,vr,IC,Long-distance,,True,False,277557915548,REGULAR,2020-11-06T10:17:00.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
2,8166,2021-03-29,10,vr,HL,Commuter,A,False,False,277557831000,REGULAR,2021-02-19T10:43:08.000Z,"[{'stationShortCode': 'LPV', 'stationUICCode':..."


#### Tehtävä 3. Poista kaikki muut sarakkeet paitsi `timeTableRows` ja `trainNumber`

In [4]:
drop_list = ["departureDate", "operatorUICCode", "operatorShortCode", "trainType", "trainCategory", "commuterLineID", "runningCurrently", "cancelled", "version", "timetableType", "timetableAcceptanceDate"]

df.drop(drop_list, axis=1, inplace = True)

df.head()

,trainNumber,timeTableRows
0,68,"[{'stationShortCode': 'KUO', 'stationUICCode':..."
1,265,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
2,8166,"[{'stationShortCode': 'LPV', 'stationUICCode':..."
3,8177,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
4,8370,"[{'stationShortCode': 'KLH', 'stationUICCode':..."


#### Tehtävä 4. Tämän jälkeen pura `timeTableRows` sarakkeen arvot jokaisesta junasta erikseen (sarakkeen arvo jokaiselle junalle sisältää useampia JSON-objekteja) ja lisää dataan mukaan junan tiedot yksilöivä `trainNumber`

In [5]:
# 
big_df = pd.DataFrame()

    
for index, row in df.iterrows():
        num = row["trainNumber"]    
        
        data = pd.DataFrame.from_dict(row["timeTableRows"])
            
        data["trainNumber"] = num
        
        big_df = pd.concat([big_df, data], axis = 0).reset_index(drop=True)
    

big_df.head(5)


,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,actualTime,differenceInMinutes,causes,trainReady,liveEstimateTime,estimateSource,trainNumber
0,KUO,408,FI,DEPARTURE,True,True,1,False,2021-03-29T11:20:00.000Z,2021-03-29T11:20:17.000Z,0.0,[],"{'source': 'KUPLA', 'accepted': True, 'timesta...",NaN,NaN,68
1,KRM,406,FI,ARRIVAL,False,NaN,,False,2021-03-29T11:31:00.000Z,2021-03-29T11:31:34.000Z,1.0,[],NaN,NaN,NaN,68
2,KRM,406,FI,DEPARTURE,False,NaN,,False,2021-03-29T11:31:00.000Z,2021-03-29T11:31:34.000Z,1.0,[],NaN,NaN,NaN,68
3,ARL,869,FI,ARRIVAL,False,NaN,,False,2021-03-29T11:35:00.000Z,2021-03-29T11:35:16.000Z,0.0,[],NaN,NaN,NaN,68
4,ARL,869,FI,DEPARTURE,False,NaN,,False,2021-03-29T11:35:00.000Z,2021-03-29T11:35:16.000Z,0.0,[],NaN,NaN,NaN,68


#### Tehtävä 5. Pudota sitten seuraavat sarakkeet datasta: `causes`, `estimateSource`, `liveEstimateTime`, `differenceInMinutes`, `commercialStop` ja `trainReady`

In [6]:
drop_list = ["causes", "estimateSource", "liveEstimateTime", "differenceInMinutes", "commercialStop", "trainReady"]

big_df.drop(drop_list, axis=1, inplace = True)
big_df.tail()

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,actualTime,trainNumber
1661,ILR,1030,FI,ARRIVAL,True,711,False,2021-03-29T15:57:00.000Z,NaN,61984
1662,HKI,1,FI,DEPARTURE,True,4,False,2021-03-29T15:54:00.000Z,2021-03-29T15:53:50.000Z,64125
1663,PSL,10,FI,ARRIVAL,False,7,False,2021-03-29T15:59:00.000Z,NaN,64125
1664,PSL,10,FI,DEPARTURE,False,7,False,2021-03-29T15:59:00.000Z,NaN,64125
1665,ILR,1030,FI,ARRIVAL,True,801,False,2021-03-29T16:09:00.000Z,NaN,64125


#### Tehtävä 6. Täytä jokaisen rivin sarakkeen `actualTime` puuttuvat arvot `scheduledTime` -sarakkeen arvolla.

In [7]:
big_df["actualTime"].fillna(data["scheduledTime"], inplace = True)
big_df.head(10)

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,actualTime,trainNumber
0,KUO,408,FI,DEPARTURE,True,1,False,2021-03-29T11:20:00.000Z,2021-03-29T11:20:17.000Z,68
1,KRM,406,FI,ARRIVAL,False,,False,2021-03-29T11:31:00.000Z,2021-03-29T11:31:34.000Z,68
2,KRM,406,FI,DEPARTURE,False,,False,2021-03-29T11:31:00.000Z,2021-03-29T11:31:34.000Z,68
3,ARL,869,FI,ARRIVAL,False,,False,2021-03-29T11:35:00.000Z,2021-03-29T11:35:16.000Z,68
4,ARL,869,FI,DEPARTURE,False,,False,2021-03-29T11:35:00.000Z,2021-03-29T11:35:16.000Z,68
5,SLN,405,FI,ARRIVAL,False,,False,2021-03-29T11:40:00.000Z,2021-03-29T11:39:22.000Z,68
6,SLN,405,FI,DEPARTURE,False,,False,2021-03-29T11:40:00.000Z,2021-03-29T11:39:22.000Z,68
7,SNJ,404,FI,ARRIVAL,True,1,False,2021-03-29T11:47:00.000Z,2021-03-29T11:46:26.000Z,68
8,SNJ,404,FI,DEPARTURE,True,1,False,2021-03-29T11:48:00.000Z,2021-03-29T11:48:26.000Z,68
9,MRK,896,FI,ARRIVAL,True,002,False,2021-03-29T11:56:00.000Z,2021-03-29T11:55:54.000Z,68


## Vaihe 2

#### Tehtävä 7. Pura tämän kansion sisällä olevan `data.html` -tiedoston `table`-elementin solujen arvot käyttäen regex-syntaksia ja yhdistä ne vaiheessa 1 luotuun dataframeen. Huom: tehtävässä tulee käyttää vain regexiä (Esim. Pandasin read_html()-metodin käyttö on kielletty)!

In [37]:
import re
file = open("data.html", "r")
lines = file.readlines()

listToStr = ' '.join(map(str, lines))
  
#print(listToStr)
re_pattern = r'table.*'

re_search = re.findall(re_pattern, listToStr)

print(re_search)

file.close()


['table, th, td {', 'table{', 'table>', 'table>']


## Vaihe 3

#### Tehtävä 8. Tee SQLite tietokanta ja lisää sinne vaiheessa 2 luotu dataframe.


In [9]:
import sqlite3
db_connection = sqlite3.connect("trains.db")
c = db_connection.cursor()

big_df.to_sql(name = "trains", con = db_connection, if_exists = "fail", index = False)

db_connection.commit()
db_connection.close()